In [3]:
import numpy as np
from pyorbit.subroutines import kepler_exo

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import collections
import matplotlib.gridspec as gridspec # GRIDSPEC !


In [4]:
def plots_in_grid():
    # Partially taken from here:
    # http://www.sc.eso.org/~bdias/pycoffee/codes/20160407/gridspec_demo.html
    
    gs = gridspec.GridSpec(2,1, height_ratios=[3.0,1.0])
    # Also make sure the margins and spacing are apropriate
    gs.update(left=0.2, right=0.95, bottom=0.08, top=0.93, wspace=0.02, hspace=0.03)
    
    ax_0 = plt.subplot(gs[0])
    ax_1 = plt.subplot(gs[1])

    # Adding minor ticks only to x axis
    from matplotlib.ticker import AutoMinorLocator
    minorLocator = AutoMinorLocator()
    ax_0.xaxis.set_minor_locator(minorLocator)
    ax_1.xaxis.set_minor_locator(minorLocator)

    # Disabling the offset on top of the plot
    ax_0.ticklabel_format(useOffset=False, style='plain')
    ax_1.ticklabel_format(useOffset=False, style='plain')
    return ax_0, ax_1

In [41]:
# Path to the configuration file, It can be absolute or relative with respect to this jupyter notebook
path_to_configuration_file = './'    

 # Change according to the name of the configuration file
configuration_name = 'RV_GPtrained_2p'

# Sampler used in the analysis
sampler = 'emcee'

# Set of parameters extracted from the posteriors
# it can be 'model_files', 'MAP_model_files', 'sampleMED_model_files'
solution_set = 'model_files' 

# First part of the output file name
output_name = 'RV_GPtrained_2p' 

# List the datasets as they appear in the configuration file (e.g., RV_data)
# and the name you want to use in the label (e.g., HARPS-N)
# include the activity indexes
datasets_dictionary = {
    'RVdata': 'HARPS and HARPS-N',
    'BISdata': 'Bisector span',
    'Sdata': 'S index',
    
}

# name of the activty model to be displayed
activity_model = 'gp_quasiperiodic'

rv_datasets= ['RVdata']

# Split the dataset if a jitter/offset flag is provided
# set split_rv_datasets_dictionary = None if jitter/offset flag have not been used for any dataset
# otherwise, just specify the name of the dataset as 'jitter_0' argument
# This dictionary will replace rv_datasets_dictionary if the former is not None
split_datasets_dictionary = {
    'RVdata': {
        'jitter_0': 'HARPS-N',
        'jitter_1': 'HARPS',
    },
    'BISdata': {
        'jitter_0': 'HARPS-N',
        'jitter_1': 'HARPS',
    },
    'Sdata': {
        'jitter_0': 'HARPS-N',
        'jitter_1': 'HARPS',
    },

}

# The script will generate a plot for each planet in planet_dictionary
planet_dictionary = collections.OrderedDict()

# Name the planet in the same way as in the configuration file 
# - Period and Tc (fitted or derived) must be taken form the output of pyorbit_results
# - limits_folded_x  will define the horizontal range of the plot, in orbital phase 
# - limits_folded_y  will define the vertical range of the plot showing the radial velocities 
#   of the planet (i.e., observations minus any other model) and the model
# - limits_residuals_y will define the vertical range of the residuals plot 
# - transit_folded: if True, RVs are folded according to the provided time of conjuction Tc (orbital phase=0)
#   if False, RVs are folded starting from the reference time provided in the configuration file

planet_dictionary['b'] =  {
    'Period': 0.280324958,   # Orbital period of the planet 
    'Tc': 57744.07151, # Central time of transit (fitted or derived)
    'limits_folded_x': [-0.75, 0.75],
    'limits_folded_y': [-10.8, 10.8],
    'limits_residuals_y': [-9.5, 9.5],
    'transit_folded': True
}

planet_dictionary['c'] =  {
    'Period': 7.748994,
    'Tc': 58371.07415,
    'limits_folded_x': [-0.25, 1.25],
    'limits_folded_y': [-10.8, 10.8],
    'limits_residuals_y': [-9.5, 9.5],
    'transit_folded': False
}

# general_dictionary provides the values for the unfolded RV plot 
# (all observed RVs after correcting for offsets)
# and other keywords used in the plot 

general_dictionary = {
    'Tref': 59200.,
    'add_jitter': True,
    'reference_planet': 'b',
    'font_label': 16,
    'dot_size': 18, 
    'figsize': (8,8),
    'plot_keplerians': False,
    'limits_bjd': [57975, 58125],
    'limits_RVdata_y': [-27, 22],
    'limits_RVdata_residuals': [-14, 14],
    
    
    #'limits_full_x': [-0.75, 0.75],
    #'limits_full_y': [-69.0, 69.0],
    #'limits_residuals_y': [-5.5, 5.5],
}

#general_dictionary['planet_color_code'] = True
general_dictionary['planet_color_code'] = False


In [42]:
# Folder RV plot for each of the planet

plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']
matplotlib.rcParams.update({'font.size': general_dictionary['font_label']})

n_color = 0

#set equal to zero if the results have been obtained with Version 10 of PyORBIT
skip_header = 0

dot_size = general_dictionary['dot_size'] 
dir_plot = path_to_configuration_file + '/' + configuration_name + '/' + sampler + '_plot/'+solution_set+'/'
dir_planet_samples = path_to_configuration_file + '/' + configuration_name + '/' + sampler + '_plot/planet_samples/'
dir_all_samples = path_to_configuration_file + '/' + configuration_name + '/' + sampler + '_plot/all_samples/'    
    
    
  
if split_datasets_dictionary:
    working_datasets_dictionary = split_datasets_dictionary
else:
    working_datasets_dictionary = {}
    for dataset_name, dataset_label in split_datasets_dictionary.items():
        working_datasets_dictionary[dataset_name] = {'jitter_0': dataset_label}  


for dataset_name, dataset_dict in working_datasets_dictionary.items():

    
    fig = plt.figure(figsize=general_dictionary['figsize'])
    ax_0, ax_1 = plots_in_grid()
    
    if dataset_name in rv_datasets and general_dictionary.get('plot_keplerians', False):
    
        for n_planet, planet_name in enumerate(planet_dictionary):
            rv_keplerian = np.genfromtxt(dir_plot + 'RV_planet_'+planet_name +'_kep.dat')


            if n_planet==0:
                rv_keplerian_bjd = rv_keplerian[:,0]*1.
                rv_keplerian_sum = rv_keplerian[:,1]*1.
            else:
                rv_keplerian_sum += rv_keplerian[:,1]

        # Plot of the model
        ax_0.plot(rv_keplerian_bjd,rv_keplerian_sum, color='C9', linestyle='-', zorder=2, label='RV model')
    

    activity_full = np.genfromtxt(dir_plot + dataset_name + '_' + activity_model +'_full.dat')
    activity_data = np.genfromtxt(dir_plot + dataset_name + '_' + activity_model +'.dat')
    
    ax_0.fill_between(activity_full[:,0], 
                      activity_full[:,-2]-activity_full[:,-1], 
                      activity_full[:,-2]-activity_full[:,-1], 
                      alpha=0.25, color='k')

    
    
    if general_dictionary['add_jitter']:
        activity_data_jiterr =  np.sqrt(activity_data[:,9]**2 + activity_data[:,12]**2)

    if len(dataset_dict) == 1: # no jitter/offset for this dataset
        starting_flag = -1
    else:
        starting_flag = 0

    for splitted_name, splitted_label in dataset_dict.items():

        
        default_color = 'C' + repr(n_color)

        if splitted_name[:6] == 'jitter':
            sel = (activity_data[:,-3] == starting_flag)
        elif splitted_name[:6] == 'offset':
            sel = (activity_data[:,-2] == starting_flag)
        else:
            sel = (activity_data[:,-1] == starting_flag)
        starting_flag += 1

        activity_compare = activity_data[sel,8]
        #activity_compare_err = activity_data[sel,9]
        activity_residuals = activity_data[sel,10]
        activity_bjd = activity_data[sel,0]
        activity_compare_err = activity_data_jiterr[sel]


        ax_0.errorbar(activity_bjd, activity_compare, yerr=activity_compare_err, color='black', markersize=0, alpha=0.25, fmt='o', zorder=0)
        ax_0.scatter(activity_bjd, activity_compare, c=default_color, s=dot_size, zorder=5, alpha=1.0, label=splitted_label)

        # plot of the residuals
        ax_1.errorbar(activity_bjd, activity_residuals, yerr=activity_compare_err, color='black', markersize=0, alpha=0.25, fmt='o', zorder=1)
        ax_1.scatter(activity_bjd, activity_residuals, c=default_color, s=dot_size, zorder=10, alpha=1.0)

        if not general_dictionary['planet_color_code']:
            n_color += 1

    name_limit_dataset = 'limits_' + dataset_name + '_y'
    name_limit_residuals = 'limits_' + dataset_name + '_residuals'

    ax_1.axhline(0.000, c='k', zorder=3)
        
    if general_dictionary.get('limits_bjd', False):
        ax_0.set_xlim(general_dictionary['limits_bjd'][0], general_dictionary['limits_bjd'][1])
        ax_1.set_xlim(general_dictionary['limits_bjd'][0], general_dictionary['limits_bjd'][1])
    if general_dictionary.get(name_limit_dataset, False):
        ax_0.set_ylim(general_dictionary[name_limit_dataset][0], general_dictionary[name_limit_dataset][1])
    if general_dictionary.get(name_limit_residuals, False):
        ax_1.set_ylim(general_dictionary[name_limit_residuals][0], general_dictionary[name_limit_residuals][1])

    
    ax_0.axes.get_xaxis().set_ticks([])
    
    ax_0.set_ylabel('RV [m/s]')
    ax_1.set_xlabel('Orbital Phase')
    ax_1.set_ylabel('Residuals [m/s]')
    
    if not general_dictionary['planet_color_code']:
        ax_0.legend()
    plot_filename = output_name+'_'+dataset_name+'_activity.png'
    print('Folded plot for activity index',dataset_name,' saved to:', plot_filename )
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight')

    
    if general_dictionary['planet_color_code']:
        n_color += 1
    else:
        n_color = 0
        
    plt.close()
    

Folded plot for activity index RVdata  saved to: RV_GPtrained_2p_RVdata_activity.png
Folded plot for activity index BISdata  saved to: RV_GPtrained_2p_BISdata_activity.png
Folded plot for activity index Sdata  saved to: RV_GPtrained_2p_Sdata_activity.png


In [ ]:
plt.close()

